In [1]:
#check if JupyROOT works 
import ROOT

Welcome to JupyROOT 6.15/01


In [2]:
import glob
import itertools as it
import numpy as np
import math , array 
import random
from subprocess import call
from os.path import isfile
from numpy.lib.recfunctions import append_fields
from itertools import product

import keras
import keras.optimizers
from keras.optimizers import SGD
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Activation
from keras.regularizers import l2

from ROOT import TCanvas, TMVA, TFile, TTree, TCut

Using TensorFlow backend.


In [3]:
bkg='TTV'
path='../root/2017/forBDTtraining/'
frac=0.5

In [4]:
varlist=[
    #'taupt',
    #'avg_dr_jet',
    'njet',
    'tau_eta',
    #'dr_leps',
    #'mindr_tau_jet',
    #'mindr_lep1_jet',
    #'dr_lep1_tau_jet',
    'mTauTauVis',
    'mT_lep2',
    #'mindr_lep2_jet',
    #'dr_lep2_tau_ss',
    #'dr_lep1_tau_os',
    #'mbb_loose',
    #'lep2_conePt',
    #'nBJetLoose',
    #'min_lep_eta',
    'mT_lep1',
    'ptmiss',
    #'max_lep_eta',
    #'lep1_conePt',
    #'drbb_loose'
]

In [5]:
###============================path to root files

forBDTtraining="forBDTtraining"
channel='2los_1tau'
rootfilepath = path+"*/*.root"
list = glob.glob(rootfilepath)
#print list


In [6]:
#========convert list of filenames to list of tfile objects

root_files=[]
for x in range(0,len(list)):
    try: root_files.append(TFile.Open(list[x]))
    except: continue
#root_files

In [7]:
#=================A dictionary to contain all data
##the list is: ['signal',TTW','TTZ','TT']
if bkg == "TTV":
    myDict = {
    'signal':[],
    'TTW':[],
    'TTZ':[],
    #'TT':[]
    }
elif bkg == "TT":
    myDict = {
    'signal':[],
    #'TTW':[],
    #'TTZ':[],
    'TT':[]
    }
elif bkg == "all":
    myDict = {
    'signal':[],
    'TTW':[],
    'TTZ':[],
    'TT':[]
    }
myDict

{'TTW': [], 'TTZ': [], 'signal': []}

In [8]:
#====================Filling the dictionary
nEvtTot = 0
for categories in myDict.keys():
    try:
        print "\n\nEntering processes "+categories+"\nRoot files found in category: "
        for root_file in root_files:
            path_in = channel+"_"+forBDTtraining+"/sel/evtntuple/"+categories+"/evtTree"
            evt_tree=root_file.Get(path_in)
            if (evt_tree is not None) :
                if isinstance(evt_tree,ROOT.TTree):
                    print categories+"\t"+root_file.GetName()+' '+str(evt_tree.GetEntries())+' entries.'
                    myDict[categories].append(evt_tree)
                    nEvtTot += evt_tree.GetEntries()
            else:
                continue
    except:
        continue
myDict



Entering processes TTZ
Root files found in category: 
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_1.root 3309 entries.
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_2.root 3305 entries.
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_3.root 3238 entries.
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_4.root 3170 entries.
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_5.root 3317 entries.
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_6.root 3303 entries.
TTZ	../root/2017/forBDTtraining/TTZJets_LO/TTZJets_LO_forBDTtraining_central_7.root 311 entries.


Entering processes signal
Root files found in category: 
signal	../root/2017/forBDTtraining/ttHToNonbb_M125_powheg/ttHToNonbb_M125_powheg_forBDTtraining_central_1.root 8593 entries.
signal	../root/2017/forBDTtraining/ttHToNonbb_M125_powheg/ttHToNonbb_M125_p

{'TTW': [<ROOT.TTree object ("evtTree") at 0x7fffe1ccde40>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1c70570>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1d2b0c0>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1d88d80>],
 'TTZ': [<ROOT.TTree object ("evtTree") at 0x7fffe103e720>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1aa1320>,
  <ROOT.TTree object ("evtTree") at 0x7fffe143b4e0>,
  <ROOT.TTree object ("evtTree") at 0x7fffe143a7b0>,
  <ROOT.TTree object ("evtTree") at 0x7fffe14319c0>,
  <ROOT.TTree object ("evtTree") at 0x7fffe0afc950>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1159200>],
 'signal': [<ROOT.TTree object ("evtTree") at 0x7fffe1ba8bd0>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1bde070>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1c0e620>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1c0d180>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1ca05d0>,
  <ROOT.TTree object ("evtTree") at 0x7fffe1ccec60>]}

In [9]:
#======================Setting up TMVA and Loading Data

dataloader = TMVA.DataLoader('dataset')
for branch in myDict['signal'][0].GetListOfBranches():
    for selection in varlist:
        if (selection == branch.GetName()):
            try:
                dataloader.AddVariable(selection)
            except:
                print "\n\n\n\n\n\n\n"+selection+" has problems."

TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()

output = TFile.Open('TMVA.root', 'RECREATE')
#dataloader.AddSignalTree(myDict['signal'][0], 1.0)


for categories in myDict.keys():
    if categories == 'signal': 
        for i in range(0,(len(myDict[categories]))):
            if myDict[categories][i].GetEntries()!=0:
                dataloader.AddSignalTree(myDict[categories][i], 1.0)
    else:
        for i in range(0,(len(myDict[categories]))):
            if myDict[categories][i].GetEntries()!=0:
                dataloader.AddBackgroundTree(myDict[categories][i], 1.0)
#train_sig = myDict["signal"][0].GetEntries()*frac
#print train_sig
#train_bkg = (nEvtTot - myDict["signal"][0].GetEntries())*frac
#print train_bkg
dataloader.PrepareTrainingAndTestTree(TCut(''),
                                      'nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V')


DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree evtTree of type Background with 3309 events
                         : Add Tree evtTree of type Background with 3305 events
                         : Add Tree evtTree of type Background with 3238 events
                         : Add Tree evtTree of type Background with 3170 events
                         : Add Tree evtTree of type Background with 3317 events
                         : Add Tree evtTree of type Background with 3303 events
                         : Add Tree evtTree of type Background with 311 events
DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree evtTree of type Signal with 8593 events
                         : Add Tree evtTree of type Signal with 8526 events
                         : Add Tree evtTree of type Signal with 8650 events
                         : Add Tree evtTree of type Signal with 8579 events
             

In [10]:
# ==================================Book methods
factory = TMVA.Factory('TMVAClassification', output,
                       '!V:!Silent:Color:DrawProgressBar:AnalysisType=Classification')


In [11]:
####=============PyKeras 

#====================building the model
model = Sequential()
num_layers=100
num_nodes=100
model.add(Dense(num_nodes, activation='tanh', input_dim=len(varlist)-1))
for i in range(0,num_layers):
    model.add(Dense(num_nodes, activation='tanh'))
model.add(Dense(2, activation='relu'))
# ====================Set loss and optimizer
model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['binary_accuracy', ])
# =====================Store model to file
model.save('model.h5')
model.summary()

factory.BookMethod(dataloader, TMVA.Types.kPyKeras, 'PyKeras','!H:!V:FilenameModel=model.h5:NumEpochs=100')



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
__________

<ROOT.TMVA::MethodPyKeras object ("PyKeras") at 0x7fbc2e22aec0>

Factory                  : Booking method: PyKeras
                         : 
                         : Load model from file: model.h5


2018-10-04 04:44:15.799432: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


In [12]:
factory.TrainAllMethods()

__main__:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2000 samples, validate on 72978 samples
Epoch 1/100
2000/2000 [==============================] - 11s 5ms/step - loss: 2.8586 - binary_accuracy: 0.4840 - val_loss: 0.5126 - val_binary_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 0.51257, saving model to dataset/weights/TrainedModel_PyKeras.h5
Epoch 2/100
2000/2000 [==============================] - 8s 4ms/step - loss: 14.6819 - binary_accuracy: 0.4250 - val_loss: 0.5000 - val_binary_accuracy: 0.5000

Epoch 00002: val_loss improved from 0.51257 to 0.50000, saving model to dataset/weights/TrainedModel_PyKeras.h5
Epoch 3/100
2000/2000 [==============================] - 7s 4ms/step - loss: 0.5000 - binary_accuracy: 0.5000 - val_loss: 0.5000 - val_binary_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 0.50000
Epoch 4/100
2000/2000 [==============================] - 8s 4ms/step - loss: 0.5000 - binary_accuracy: 0.5000 - val_loss: 0.5000 - val_binary_accuracy: 0.5000

Epoch 00004: val_loss did not improve 

KeyboardInterrupt: 

Exception: void TMVA::Factory::TrainAllMethods() =>
    FATAL error (C++ exception of type runtime_error)

Factory                  : Train all methods
DataSetFactory           : [dataset] : Number of events in input trees
                         : 
                         : 
                         : Number of training and testing events
                         : ---------------------------------------------------------------------------
                         : Background -- training events            : 1000
                         : Background -- testing events             : 29952
                         : Background -- training and testing events: 30952
                         : Signal     -- training events            : 1000
                         : Signal     -- testing events             : 43026
                         : Signal     -- training and testing events: 44026
                         : 
DataSetInfo              : Correlation matrix (Background):
                         : ------------------------------------------------------
                         :          